In [ ]:
!pip install wandb

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import wandb
import matplotlib.pyplot as plt
from wandb.keras import WandbCallback

In [3]:
cd /content/drive/MyDrive/MachineLearning/MachineLearning10

/content/drive/MyDrive/MachineLearning/MachineLearning10


### 1- Load and preprocessing data

In [4]:
# load data

train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')
print(train_data.shape, test_data.shape)
train_data.info()

(1460, 81) (1459, 80)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null  

### numeric_features


In [5]:

numeric_features = train_data.select_dtypes(include=[np.number])
numeric_features.dtypes

Id                 int64
MSSubClass         int64
LotFrontage      float64
LotArea            int64
OverallQual        int64
OverallCond        int64
YearBuilt          int64
YearRemodAdd       int64
MasVnrArea       float64
BsmtFinSF1         int64
BsmtFinSF2         int64
BsmtUnfSF          int64
TotalBsmtSF        int64
1stFlrSF           int64
2ndFlrSF           int64
LowQualFinSF       int64
GrLivArea          int64
BsmtFullBath       int64
BsmtHalfBath       int64
FullBath           int64
HalfBath           int64
BedroomAbvGr       int64
KitchenAbvGr       int64
TotRmsAbvGrd       int64
Fireplaces         int64
GarageYrBlt      float64
GarageCars         int64
GarageArea         int64
WoodDeckSF         int64
OpenPorchSF        int64
EnclosedPorch      int64
3SsnPorch          int64
ScreenPorch        int64
PoolArea           int64
MiscVal            int64
MoSold             int64
YrSold             int64
SalePrice          int64
dtype: object

### categoricals features

In [6]:
categoricals = train_data.select_dtypes(exclude=[np.number])
categoricals.describe().transpose()

,count,unique,top,freq
MSZoning,1460,5,RL,1151
Street,1460,2,Pave,1454
Alley,91,2,Grvl,50
LotShape,1460,4,Reg,925
LandContour,1460,4,Lvl,1311
Utilities,1460,2,AllPub,1459
LotConfig,1460,5,Inside,1052
LandSlope,1460,3,Gtl,1382
Neighborhood,1460,25,NAmes,225
Condition1,1460,9,Norm,1260


### 1-2  convert categoricals features to numeric-features


In [7]:

# condition_pivot = train_data.pivot_table(index='GarageType', values='SalePrice', aggfunc=np.median)
# condition_pivot.plot(kind='bar', color='blue')
# plt.xlabel('GarageType')
# plt.ylabel('Median Sale Price')
# plt.xticks(rotation=0)
# plt.show()

train_data = train_data.replace(['COD', 'CWD', 'Con', 'ConLD', 'ConLI', 'New', 'Oth', 'WD', 'ConLw'], [0, 1, 2, 3, 4, 5, 6, 7, 8]) #SaleType
test_data = test_data.replace(['COD', 'CWD', 'Con', 'ConLD', 'ConLI', 'New', 'Oth', 'WD', 'ConLw'], [0, 1, 2, 3, 4, 5, 6, 7, 8]) #SaleType
train_data = train_data.replace(['Abnorml', 'AdjLand', 'Alloca', 'Family', 'Normal', 'Partial'], [0, 1, 2, 3, 4, 5]) #SaleCondition
test_data = test_data.replace(['Abnorml', 'AdjLand', 'Alloca', 'Family', 'Normal', 'Partial'], [0, 1, 2, 3, 4, 5]) #SaleCondition
train_data = train_data.replace(['C (all)', 'FV', 'RH', 'RL', 'RM'], [0, 1, 2, 3, 4]) #MSZoning
test_data = test_data.replace(['C (all)', 'FV', 'RH', 'RL', 'RM'], [0, 1, 2, 3, 4]) #MSZoning
train_data = train_data.replace(['Grvl', 'Pave'], [0, 1]) # Alley, street
test_data = test_data.replace(['Grvl', 'Pave'], [0, 1]) # Alley, street
train_data = train_data.replace(['BrkCmn', 'BrkFace', 'CBlock', 'Stone', 'None'], [0, 1, 2, 3, 4]) # MasVnrType
test_data = test_data.replace(['BrkCmn', 'BrkFace', 'CBlock', 'Stone', 'None'], [0, 1, 2, 3, 4]) # MasVnrType
train_data = train_data.replace(['Ex', 'Fa', 'Gd', 'Po', 'TA'], [0, 1, 2, 3, 4]) # HeatingQC
test_data = test_data.replace(['Ex', 'Fa', 'Gd', 'Po', 'TA'], [0, 1, 2, 3, 4]) # HeatingQC

# print (train_data.Street.value_counts())


### 1-3- calculate correlation between numerical features and labels


In [8]:
numeric_features = train_data.select_dtypes(include=[np.number])
corr = numeric_features.corr()
corr['SalePrice'].sort_values(ascending=False)


SalePrice        1.000000
OverallQual      0.790982
GrLivArea        0.708624
GarageCars       0.640409
GarageArea       0.623431
TotalBsmtSF      0.613581
1stFlrSF         0.605852
FullBath         0.560664
Alley            0.534319
TotRmsAbvGrd     0.533723
YearBuilt        0.522897
YearRemodAdd     0.507101
GarageYrBlt      0.486362
MasVnrArea       0.477493
Fireplaces       0.466929
BsmtFinSF1       0.386420
LotFrontage      0.351799
WoodDeckSF       0.324413
2ndFlrSF         0.319334
OpenPorchSF      0.315856
HalfBath         0.284108
LotArea          0.263843
BsmtFullBath     0.227122
BsmtUnfSF        0.214479
SaleCondition    0.213092
BedroomAbvGr     0.168213
GarageCond       0.150576
ExterCond        0.117303
GarageQual       0.115119
ScreenPorch      0.111447
PoolArea         0.092404
BsmtCond         0.060362
MoSold           0.046432
3SsnPorch        0.044584
Street           0.041036
BsmtFinSF2      -0.011378
BsmtHalfBath    -0.016844
MiscVal         -0.021190
Id          

### 1-4- selected features


In [9]:

X_train = train_data[['OverallQual', 'GrLivArea', 'GarageCars', 
                      'GarageArea', 'TotalBsmtSF', '1stFlrSF', 'FullBath',
                      'Alley', 'TotRmsAbvGrd', 'YearBuilt',
                      'YearRemodAdd', 'GarageYrBlt', 'MasVnrArea',
                      'Fireplaces', 'BsmtFinSF1',       
                      'LotFrontage', 'WoodDeckSF', '2ndFlrSF',
                      'OpenPorchSF', 'HalfBath', 'LotArea', 
                      'BsmtFullBath', 'BsmtUnfSF', 'SaleCondition',       
                      'BedroomAbvGr', 'GarageCond', 'ExterCond',
                      'GarageQual','ScreenPorch', ]]  
Y_train = train_data[['SalePrice']]
Y_train = np.log(Y_train)

X_test = test_data[['OverallQual', 'GrLivArea', 'GarageCars', 
                      'GarageArea', 'TotalBsmtSF', '1stFlrSF', 'FullBath',
                      'Alley', 'TotRmsAbvGrd', 'YearBuilt',
                      'YearRemodAdd', 'GarageYrBlt', 'MasVnrArea',
                      'Fireplaces', 'BsmtFinSF1',       
                      'LotFrontage', 'WoodDeckSF', '2ndFlrSF',
                      'OpenPorchSF', 'HalfBath', 'LotArea', 
                      'BsmtFullBath', 'BsmtUnfSF', 'SaleCondition',       
                      'BedroomAbvGr', 'GarageCond', 'ExterCond',
                      'GarageQual','ScreenPorch', ]]   

Y_test = pd.read_csv('data/sample_submission.csv')
Y_test = Y_test[['SalePrice']]
Y_test = np.log(Y_test)

print(X_test.shape, Y_test.shape)
print(X_train.shape, Y_train.shape)

(1459, 29) (1459, 1)
(1460, 29) (1460, 1)


### null columns in X_train


In [10]:

nulls = pd.DataFrame(X_train.isnull().sum().sort_values(ascending=False)[:10])
nulls.columns = ['Null Count']
nulls.index.name = 'Feature'
nulls

,Null Count
Feature,
Alley,1369
LotFrontage,259
GarageCond,81
GarageQual,81
GarageYrBlt,81
MasVnrArea,8
ScreenPorch,0
GrLivArea,0
GarageCars,0


### null columns in X_test


In [11]:

nulls = pd.DataFrame(X_test.isnull().sum().sort_values(ascending=False)[:10])
nulls.columns = ['Null Count']
nulls.index.name = 'Feature'
nulls

,Null Count
Feature,
Alley,1352
LotFrontage,227
GarageYrBlt,78
GarageCond,78
GarageQual,78
MasVnrArea,15
BsmtFullBath,2
TotalBsmtSF,1
BsmtFinSF1,1


### 1-5- fill null columns


In [ ]:

X_train['Alley'] = X_train['Alley'].fillna(2)
X_train['LotFrontage'] = X_train['Alley'].fillna(np.round(np.mean(X_train['LotFrontage']), 0))
X_train['GarageCond'] = X_train['GarageCond'].fillna(5)
X_train['GarageQual'] = X_train['GarageQual'].fillna(5)
X_train['GarageYrBlt'] = X_train['GarageYrBlt'].fillna(np.round(np.mean(X_train['GarageYrBlt']), 0))
X_train['MasVnrArea'] = X_train['MasVnrArea'].fillna(np.round(np.mean(X_train['MasVnrArea']), 0))

X_test['Alley'] = X_test['Alley'].fillna(2)
X_test['LotFrontage'] = X_test['Alley'].fillna(np.round(np.mean(X_test['LotFrontage']), 0))
X_test['GarageCond'] = X_test['GarageCond'].fillna(5)
X_test['GarageQual'] = X_test['GarageQual'].fillna(5)
X_test['GarageYrBlt'] = X_test['GarageYrBlt'].fillna(np.round(np.mean(X_test['GarageYrBlt']), 0))
X_test['MasVnrArea'] = X_test['MasVnrArea'].fillna(np.round(np.mean(X_test['MasVnrArea']), 0))
X_test['BsmtFullBath'] = X_test['BsmtFullBath'].fillna(4)
X_test['BsmtFinSF1'] = X_test['BsmtFinSF1'].fillna(np.round(np.mean(X_test['BsmtFinSF1']), 0))
X_test['TotalBsmtSF'] = X_test['TotalBsmtSF'].fillna(np.round(np.mean(X_test['TotalBsmtSF']), 0))
X_test['BsmtUnfSF'] = X_test['BsmtUnfSF'].fillna(np.round(np.mean(X_test['BsmtUnfSF']), 0))
X_test['GarageArea'] = X_test['GarageArea'].fillna(np.round(np.mean(X_test['GarageArea']), 0))
X_test['GarageCars'] = X_test['GarageCars'].fillna(5)


In [13]:
X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_test = np.array(X_test)
Y_test = np.array(Y_test)
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

(1460, 29) (1460, 1) (1459, 29) (1459, 1)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

In [ ]:
wandb.init(project='MLP-House Prices', entity='nahid-ebrahimian')

### creat model and ttraining

In [76]:
model = tf.keras.models.Sequential([
                        tf.keras.layers.Dense(29, input_dim = 29, activation='relu'),
                        tf.keras.layers.Dense(64, activation='relu'),
                        tf.keras.layers.Dense(128, activation='relu'),
                        tf.keras.layers.Dense(256, activation='relu'),
                        tf.keras.layers.Dense(512, activation='relu'), 
                        tf.keras.layers.Dense(1, activation='linear')
                                    ])

In [77]:
config = wandb.config
config.learning_rate = 0.001

# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=config.learning_rate),
#               loss=tf.keras.losses.MAE)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.MAE)


In [78]:
model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=100, 
callbacks=[WandbCallback()])

Epoch 1/100
7/7 [==============================] - 1s 30ms/step - loss: 248.6100 - val_loss: 241.5672
Epoch 2/100
7/7 [==============================] - 0s 11ms/step - loss: 134.5993 - val_loss: 69.1400
Epoch 3/100
7/7 [==============================] - 0s 8ms/step - loss: 50.4622 - val_loss: 9.7500
Epoch 4/100
7/7 [==============================] - 0s 8ms/step - loss: 21.7747 - val_loss: 14.3642
Epoch 5/100
7/7 [==============================] - 0s 8ms/step - loss: 11.5073 - val_loss: 4.3586
Epoch 6/100
7/7 [==============================] - 0s 8ms/step - loss: 7.8026 - val_loss: 3.4445
Epoch 7/100
7/7 [==============================] - 0s 9ms/step - loss: 6.6741 - val_loss: 5.3413
Epoch 8/100
7/7 [==============================] - 0s 8ms/step - loss: 4.3300 - val_loss: 2.8115
Epoch 9/100
7/7 [==============================] - 0s 8ms/step - loss: 5.6389 - val_loss: 4.6570
Epoch 10/100
7/7 [==============================] - 0s 8ms/step - loss: 3.2911 - val_loss: 3.1941
Epoch 11/100
7/7

In [79]:
loss = model.evaluate(X_test, Y_test)
np.exp(loss)

46/46 [==============================] - 0s 2ms/step - loss: 0.6222


1.863028759840387